### 1b

In [2]:
import sys
!{sys.executable} -m pip install numpy
import sys
!{sys.executable} -m pip install surprise

from surprise import Dataset, SlopeOne, accuracy
from surprise.model_selection import train_test_split
from collections import defaultdict

data = Dataset.load_builtin("ml-100k")
trainset, testset= train_test_split(data, test_size=.30)

algo= SlopeOne()
algo.fit(trainset)

predictions= algo.test(testset)

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621

[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python3.9 -m pip install --upgrade pip
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will n

Calculate the RMSE

In [15]:
accuracy.rmse(predictions, verbose="true")

RMSE: 0.9477


0.9476578603608544

### 1c:

* Random splitting: Testset 20% Trainset 80%, weil man häufig 20-30% zum Testen und 70-80% verwendet (https://scholarworks.utep.edu/cs_techrep/1209/) <br><br>
* RMSE: 
    * Vorteil, dass große Fehler stärker bestraft werden
    * Man sieht besser ob das Modell viele Ausreißer hat, um diese zu reduzieren
    * MAE zu einfach um Gesamtleistung des Modells zu verstehen
                                                                   

### 1d:

In [4]:
def get_top_n(predictions, n=10):
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est, true_r))

    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

In [5]:
top_n = get_top_n(predictions, n=10)

#get the user 310 with the recommended films
#print(top_n.get('310'))
def getUserAndMovie(user):
    movie = []
    for i in top_n.get(user):
        movie.append(i[0])
    #print("User {} has the .".format(user))
    print("User {} werden folgende Filme vorgeschlagen: {}".format(user,movie))

getUserAndMovie('20')


User 20 werden folgende Filme vorgeschlagen: ['22', '210', '69', '204', '423', '378', '274', '118', '94', '252']


Get all Recommendations for all users

In [6]:
def getAllRecommendatios(predictions):
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est, true_r))

    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings
    return top_n

Get all recommendations for a single user

In [7]:
def getRecommendations(user):
    return getAllRecommendatios(predictions).get(user)

print(getRecommendations('20'))

[('22', 3.564153676698129, 5.0), ('210', 3.36244635643561, 4.0), ('69', 3.2728598907785185, 1.0), ('204', 3.2603642699937283, 3.0), ('423', 3.193303459010281, 2.0), ('378', 3.080074257285008, 3.0), ('274', 2.773948413969998, 4.0), ('118', 2.521080941185077, 4.0), ('94', 2.474361414040916, 2.0), ('252', 2.1691555586020193, 4.0), ('820', 2.157641786042335, 2.0), ('678', 1.9483057827760566, 4.0), ('243', 1.7199587197784834, 4.0), ('931', 1.414324391316733, 1.0)]


### 1: e

Zuerst haben wir ein Dictionary erstellt, welches mit der UserID, dem geschätzten Wert und dem tatsächlichen Werten befüllt wurde. Anschließend wurde die Liste mit den vorgeschlagenen Filmen absteigend nach dem Wert der recommendation sortiert. All zweite Funktion wurde eine Liste mit allen Usern und deren Filmen erstellt. Nun können wir für einen bestimmten User alle Filme mit den dazugehörigen Werten (tatsächliches Rating, ausgerechneter Wert) ausgeben.

### 2a: Calculate precision

Aufgabe 2 – Implementierung von Precision und Recall
a) Implementieren Sie eine Funktion, um die Precision basierend auf Ihrem Testest aus Aufgabe 1 zu berechnen.

In [8]:
import math
def isItemRelevant(item):
    if item[2]>=3:
        return True
    else:
        return False

def calcPrecision(rec):
    relevantScore = 0
    for i in range(math.floor(len(rec)/2)):
        if isItemRelevant(rec[i]):
            relevantScore+=1

    print("Items relevant: {}".format(relevantScore))
    print("Items recommended: {}".format(math.floor(len(rec)/2)))
    return relevantScore/math.floor(len(rec)/2)

print("Precision: {}".format(calcPrecision(getRecommendations('20'))))



Items relevant: 5
Items recommended: 7
Precision: 0.7142857142857143


### 2: b
 Implementieren Sie eine Funktion, um den Recall basierend auf Ihrem Testest aus Aufgabe 1 zu berechnen. Welche Annahmen haben Sie dafür getroffen?

In [9]:
def isItemRelevant(item, x=3):
    if item[2]>=x:
        return True
    else:
        return False

def calcRecall(rec):
    relevantScore = 0
    allRelevantScore = 0
    for i in range(math.floor(len(rec)/2)):
        if isItemRelevant(rec[i]):
            relevantScore+=1

    for i in range(len(rec)):
        if isItemRelevant(rec[i]):
            allRelevantScore+=1

    print("Items recommended relevant: {}".format(relevantScore))
    print("All relevant items: {}".format(allRelevantScore))
    return relevantScore/allRelevantScore

print("Recall: {}".format(calcRecall(getRecommendations('20'))))

Items recommended relevant: 5
All relevant items: 9
Recall: 0.5555555555555556


### 3: c

In [10]:
def precision_recall_at_k(predictions, k=10, threshold=3.5):

    user_est_true = defaultdict(list)
    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))

    precisions = dict()
    recalls = dict()
    for uid, user_ratings in user_est_true.items():

        user_ratings.sort(key=lambda x: x[0], reverse=True)

        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)

        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])

        n_rel_and_rec_k = sum(
            ((true_r >= threshold) and (est >= threshold))
            for (est, true_r) in user_ratings[:k]
        )

        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 0

        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 0

    return precisions, recalls


In [11]:
precisions, recalls =  precision_recall_at_k(predictions)
print(sum(prec for prec in precisions.values()) / len(precisions))
print(sum(rec for rec in recalls.values()) / len(recalls))

0.7209139187665161
0.47283200998455716


Uns fällt auf, dass die Werte für Precision und Recall sehr ähnlich zu unseren Werten sind. 

### 3:
a) Wiederholen Sie die Experimente aus Aufgabe 1 und Aufgabe 2 mit dem Book-Crossing Dataset3. Sie können auch gerne einen anderen Algorithmus dafür anwenden.

In [35]:
import os
from surprise import Dataset, SVD
from surprise.model_selection import KFold
from surprise import Dataset, Reader, SlopeOne
from surprise.model_selection import train_test_split
# path to dataset file
file_path = os.path.expanduser("./BX-Book-Ratings.csv")

reader = Reader(line_format="user item rating", sep=";",rating_scale=(1,10))

data2 = Dataset.load_from_file(file_path, reader=reader)

kf = KFold(n_splits=5)
algo2 = SVD()

for trainset2, testset2 in kf.split(data2):
    algo2.fit(trainset2)
    predictions2 = algo.test(testset2)

In [46]:
accuracy.rmse(predictions, verbose="true")


RMSE: 3.5048


3.504789015151917

In [37]:
top_n2 = get_top_n(predictions, n=10)

#get the user 310 with the recommended films
#print(top_n2.get('300'))
def getUserAndBook(user):
    book = []
    for i in top_n2.get(user):
        book.append(i[0])
    #print("User {} has the .".format(user))
    print("User {} werden folgende Bücher vorgeschlagen: {}".format(user,book))

getUserAndBook('300')

User 300 werden folgende Bücher vorgeschlagen: ['0449912302', '038081840X', '0312963009']


In [38]:
def getAllBookRecommendatios(predictions):
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est, true_r))

    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings
    return top_n

In [49]:
def getBookRecommendations(user):
    return getAllBookRecommendatios(predictions2).get(user)

print(getBookRecommendations('276939'))

[('2702400612', 5, 7.0), ('2253144452', 5, 9.0), ('2253152072', 5, 9.0), ('8484601072', 1.9759995229945346, 0.0), ('059035342X', 1, 0.0)]


In [51]:
print("Precision: {}".format(calcPrecision(getBookRecommendations('276939')),x=6))

Items relevant: 2
Items recommended: 2
Precision: 1.0


In [53]:
print("Recall: {}".format(calcRecall(getBookRecommendations('276939')),x=6))

Items recommended relevant: 2
All relevant items: 3
Recall: 0.6666666666666666
